# Projeto 1 - Classificador Naive-Bayes com Tweets


## Objetivo

* Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa selecionealgumas mensagens disponíveis no Twitter que mencionam esse produto e classifique-as como"relevante" ou "irrelevante".

* Com isso, essa empresa deseja que mensagens relevantes, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.

* Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamentedo Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails, porexemplo. Esse classificador permite calcular qual a probabilidade de uma mensagem ser relevante dada as palavras em seu conteúdo.

* Para realizar o MVP (minimum viable product) do projeto, você precisa implementar umaversão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.

* Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Integrantes do Projeto

**Nome:** Wilgner Lopes Araujo

**Nome:** Fabricio Neri Lima

## Informações relevantes

* Esse arquivo representa a ultima etapa do projeto onde haverá todas as etapas de cleaning da base de dados e implantação do algoritmo de classificação

* O arquivo que pesquisa os tweets esta na pasta Arquivo Jupyter/Projeto1_Obtenção_dos_tweets nessa mesma raiz do repositório

* A base de dados se encontra na pasta dados/Valorant.xlsx

* A conta utiliza no twitter para obtenção de dados é @AnalyticsInsper

* O produto escolhido para o projeto é : **Valorant**

* Os criterios para a classificação dos tweets na primeira etapa do projeto foram: Os tweets que falavam de alguma forma positiva ou negativa sobre o jogo ou até mesmo que relacionavam o jogo a alguma emoção, foram classificados como relevantes. Os demais foram classificados como Irrelevantes.


# Começando

## Declarando as bibliotecas

In [1]:
import math
import numpy as np
import pandas as pd
import re
import functools
import operator
import emoji

## Abrindo a base de dados e visualizando os primeiros 5 termos

In [2]:
dados = pd.read_excel('../dados/Valorant.xlsx')
dados.head()

,Treinamento,Etiquetas,Classificado
0,@caos__0 eu no valorant :v\nfon,2,Irrelevante
1,o cara q pra mim é o melhor player de valorant...,2,Irrelevante
2,mano algm upa meu passe no valorant na humildade,2,Irrelevante
3,"deletei o valorant, agora meu pc volta a ligar...",1,Relevante
4,quem tiver pc baixa valorant nessa porr que eu...,1,Relevante


## Informações do DataFrame

In [3]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Treinamento   500 non-null    object
 1   Etiquetas     500 non-null    int64 
 2   Classificado  500 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.8+ KB


## Classificando 

* A coluna Treinamento e Classificado são categoricas, pois ambas são variaveis qualitativas nominais

In [4]:
dados['Treinamento'] = dados['Treinamento'].astype('category')
dados['Classificado'] = dados['Classificado'].astype('category')

## Iniciando as coletas de probabilidades

### Analisando frequencia de Relevante e Irrelevante na base de dados

In [5]:
Probabilidade_total = dados.Classificado.value_counts(True)
Probabilidade_total

Irrelevante    0.542
Relevante      0.458
Name: Classificado, dtype: float64

## Declarando as variaveis de probabilidade

* Considere P(R) para probabilidade de ser Relevante
* Considere P(R_c) para probabilidade de ser Irrelevante

In [6]:
P_R_c = Probabilidade_total[0]
P_R = Probabilidade_total[1]

## Antes de prosseguir vamos limpar o codigo

* Será removido os seguintes simbolos: \n, :, ", ', (,), ., ;, :, ?, !, <, >, 

In [7]:
def limpa_texto(text):
    
    #import string
    punctuation = '[!-.:?;@_,<>/\)(#%$|\n]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

## Alem disso vamos espaçar os emojis entre si

* O codigo utilizado foi extraido e modificado do seguinte site: https://stackoverflow.com/questions/49921720/how-to-split-emoji-from-each-other-python

* O codigo utiliza a biblioteca emoji, functools, operator para atingir o objetivo de separar os emojis

In [8]:
def separa_emoji(text):
    frase = text
    frase_split_emoji = emoji.get_emoji_regexp().split(frase)
    frase_split_com_espaço_em_branco = [substr.split() for substr in frase_split_emoji]
    frase_split = functools.reduce(operator.concat, frase_split_com_espaço_em_branco)
    frase_espacada = ' '.join(frase_split)
    return frase_espacada

## Aplicando a função de limpeza na base de dados

In [9]:
dados['Treinamento'] = dados['Treinamento'].apply(limpa_texto)
dados['Treinamento'] = dados['Treinamento'].apply(separa_emoji)

In [23]:
# Checando se funcionou e se os emojis ainda funcionam
pd.set_option('max_colwidth', None)
dados.tail(25)

,Treinamento,Etiquetas,Classificado
475,rt vinibq novidade quentíssima para as lives agora sim meu subs estarão trajados para assistir gameplay de valorantrebecachuffi…,2,Irrelevante
476,sempre q eu jogo normal gamezin no valorant sempre tem um cara mt baludo no outro time,1,Relevante
477,rt diogogmachadof valorant é um jogo de corno,1,Relevante
478,thereishai vamos mais tarde um valorant,2,Irrelevante
479,nichoneves salvitheminer notisxbella eu jogo mas vc tem q ir uma de valorant cmg 😎 🍃,2,Irrelevante
480,larguei fortnite fdc valorant eh mt melhor,1,Relevante
481,segue aí conta diretamente à valorant,2,Irrelevante
482,hwllraiserz amigo saí dessa vida só vou se for valorant,2,Irrelevante
483,1kingbr mano vários jogos ficaram off valorant e r6 tbm 🤦🏾‍♂️ 🤦🏾‍♂️,2,Irrelevante
484,victorkeos riotgamesbrasil riotgames lolegendsbr leagueoflegends o melhor é q tinha uma página no insta vendendo rp barato ai fui ver o perfil pra denunciar eles vendiam smurfs de valorant e skins prestígio tipo vei,2,Irrelevante


## Separando a base de dados em Relevantes e Irrelevantes

In [11]:
filtro_relevante = dados.Classificado == 'Relevante'
filtro_irrelevante = dados.Classificado == 'Irrelevante'

dados_relevantes = dados.loc[filtro_relevante,:]
dados_irrelevantes = dados.loc[filtro_irrelevante,:]

In [18]:
dados_relevantes.head()

,Treinamento,Etiquetas,Classificado
3,deletei o valorant agora meu pc volta a ligar em 3 segundos,1,Relevante
4,quem tiver pc baixa valorant nessa porr que eu tô no ódio e vai ser o dia todo só isso,1,Relevante
5,voltei a jogar valorant so por causa do mata mata,1,Relevante
6,viciei em valorant esse é o tweet,1,Relevante
7,zncabral free fire fez mais sucesso que o valorant só no brasil no resto do mundo quase ngm joga isso,1,Relevante


## Verificando a probabilidade relativa por palavra no conjunto de relevantes

In [27]:
frase = 0
primeira_quebra = []

for i in range(len(dados_relevantes)):
    frase = dados_relevantes.iloc[i,0]
    primeira_quebra += frase.split()

palavras_quebradas_relevantes = pd.Series(primeira_quebra)


palavras_quebradas_relevantes_relativa = palavras_quebradas_relevantes.value_counts(True)
palavras_quebradas_relevantes_relativa;

In [ ]:
Aplicando o looping de probabilidade por tweet dado que ele é relevan

In [26]:
frase_quebrada = [dados_relevantes.iloc[i,0] for i in range(len(dados_relevantes))]
lista_de_lista_de_tweets = [frase_quebrada[palavra].split() for palavra in range(len(frase_quebrada))]

prob_tweet_dado_relevante = 1
lista_proba = list()

for trecho in range(len(lista_de_lista_de_tweets)):
    for palavra in lista_de_lista_de_tweets[trecho]:
        prob_tweet_dado_relevante *= palavras_quebradas_relevantes_relativa[palavra]
    lista_proba.append(prob_tweet_dado_relevante)
    prob_tweet_dado_relevante = 1
    
lista_proba;


In [15]:
data_frame = pd.DataFrame(lista_proba, columns=['P_tweet_dado_relevante'])

data_frame['P_tweet_dado_relevante * P_R'] = data_frame['P_tweet_dado_relevante']*P_R
data_frame.head()

,P_tweet_dado_relevante,P_tweet_dado_relevante * P_R
0,2.281918e-31,1.045119e-31
1,1.862654e-49,8.530956e-50
2,1.454416e-24,6.661227e-25
3,3.121318e-17,1.429563e-17
4,1.152785e-52,5.279756e-53


## 
